In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from IPython.display import display, HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
import statsmodels.api as sm
from scipy.stats import norm


In [11]:
pd.set_option('display.max_columns', None)

# EDA and Data Cleaning

1. Провести аналіз вибраного набору даних,
2. визначити вхідні та вихідні параметри,
3. візуалізувати залежності входів та виходу,
4. детектувати аномалії, неповні зразки у даних, тощо,
5. провести кореляційний аналіз входів та виходів набору даних, виявити
взаємозалежні фактори.
6. Провести підготовку даних до подальшого використання.
7. Обраний та відфільтрований набір даних розбити на навчальну та тестову
частину (70% на навчання, 30% на тест).

House Prices - Advanced Regression Techniques
https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview

In [12]:
data_raw_test = pd.read_csv(r'D:\University\semestr_7\Machine_Learning\lab1\data\test.csv')
data_raw_train = pd.read_csv(r'D:\University\semestr_7\Machine_Learning\lab1\data\train.csv')
data_raw_sample_submission = pd.read_csv(r'D:\University\semestr_7\Machine_Learning\lab1\data\sample_submission.csv')

In [13]:
data_raw_test_sample_submission = pd.concat([data_raw_test, data_raw_sample_submission["SalePrice"]], axis=1)
data_raw = pd.concat([data_raw_train, data_raw_test_sample_submission], axis=0, ignore_index=True)

In [14]:
data_raw

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.000000
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.000000
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.000000
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.000000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,167081.220949
2915,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,164788.778231
2916,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,219222.423400
2917,2918,85,RL,62.0

In [15]:
data_raw.drop(columns=["Id"], inplace=True)

In [16]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2919 non-null   int64  
 1   MSZoning       2915 non-null   object 
 2   LotFrontage    2433 non-null   float64
 3   LotArea        2919 non-null   int64  
 4   Street         2919 non-null   object 
 5   Alley          198 non-null    object 
 6   LotShape       2919 non-null   object 
 7   LandContour    2919 non-null   object 
 8   Utilities      2917 non-null   object 
 9   LotConfig      2919 non-null   object 
 10  LandSlope      2919 non-null   object 
 11  Neighborhood   2919 non-null   object 
 12  Condition1     2919 non-null   object 
 13  Condition2     2919 non-null   object 
 14  BldgType       2919 non-null   object 
 15  HouseStyle     2919 non-null   object 
 16  OverallQual    2919 non-null   int64  
 17  OverallCond    2919 non-null   int64  
 18  YearBuil

In [17]:
data_raw["MSSubClass"] = data_raw["MSSubClass"].astype("category")

https://www.kaggle.com/code/erikbruin/house-prices-lasso-xgboost-and-a-detailed-eda#the-response-variable-saleprice
потом убрать

## Changing broken data

In [18]:
def parse_dataset_description(file_path):
    result = {}
    current_key = None
    
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            if re.match(r"^[A-Za-z0-9_]+:", line):
                current_key = line.split(":")[0].strip()
                result[current_key] = []
            
            elif current_key and re.match(r"^[A-Za-z0-9]+", line):
                value = line.split()[0].strip()
                result[current_key].append(value)

    return result
valid_dict = parse_dataset_description(r"D:\University\semestr_7\Machine_Learning\lab1\data\data_description.txt")
valid_dict


{'MSSubClass': ['20',
  '30',
  '40',
  '45',
  '50',
  '60',
  '70',
  '75',
  '80',
  '85',
  '90',
  '120',
  '150',
  '160',
  '180',
  '190'],
 'MSZoning': ['A', 'C', 'FV', 'I', 'RH', 'RL', 'RP', 'RM'],
 'LotFrontage': [],
 'LotArea': [],
 'Street': ['Grvl', 'Pave'],
 'Alley': ['Grvl', 'Pave', 'NA'],
 'LotShape': ['Reg', 'IR1', 'IR2', 'IR3'],
 'LandContour': ['Lvl', 'Bnk', 'HLS', 'Low'],
 'Utilities': ['AllPub', 'NoSewr', 'NoSeWa', 'ELO'],
 'LotConfig': ['Inside', 'Corner', 'CulDSac', 'FR2', 'FR3'],
 'LandSlope': ['Gtl', 'Mod', 'Sev'],
 'Neighborhood': ['Blmngtn',
  'Blueste',
  'BrDale',
  'BrkSide',
  'ClearCr',
  'CollgCr',
  'Crawfor',
  'Edwards',
  'Gilbert',
  'IDOTRR',
  'MeadowV',
  'Mitchel',
  'Names',
  'NoRidge',
  'NPkVill',
  'NridgHt',
  'NWAmes',
  'OldTown',
  'SWISU',
  'Sawyer',
  'SawyerW',
  'Somerst',
  'StoneBr',
  'Timber',
  'Veenker'],
 'Condition1': ['Artery',
  'Feedr',
  'Norm',
  'RRNn',
  'RRAn',
  'PosN',
  'PosA',
  'RRNe',
  'RRAe'],
 'Condition2

In [19]:

def find_invalid_values(df, valid_dict):
    invalid = {}
    
    for col, valid_vals in valid_dict.items():
        if col not in df.columns:
            continue
        
        if valid_vals:
            unique_vals = df[col].dropna().unique()

            bad_vals = [v for v in unique_vals if v not in valid_vals]
            
            if bad_vals:
                invalid[col] = bad_vals
    
    return invalid

invalid_values = find_invalid_values(data_raw, valid_dict)
invalid_values


{'MSSubClass': [60,
  20,
  70,
  50,
  190,
  45,
  90,
  120,
  30,
  85,
  80,
  160,
  75,
  180,
  40,
  150],
 'MSZoning': ['C (all)'],
 'Neighborhood': ['NAmes'],
 'BldgType': ['2fmCon', 'Duplex', 'Twnhs'],
 'OverallQual': [np.int64(7),
  np.int64(6),
  np.int64(8),
  np.int64(5),
  np.int64(9),
  np.int64(4),
  np.int64(10),
  np.int64(3),
  np.int64(1),
  np.int64(2)],
 'OverallCond': [np.int64(5),
  np.int64(8),
  np.int64(6),
  np.int64(7),
  np.int64(4),
  np.int64(2),
  np.int64(3),
  np.int64(9),
  np.int64(1)],
 'Exterior1st': ['Wd Sdng'],
 'Exterior2nd': ['Wd Shng', 'Wd Sdng', 'CmentBd', 'Brk Cmn']}

In [20]:
data_raw['MSZoning'] = data_raw['MSZoning'].replace('C (all)', 'C')
data_raw['Neighborhood'] = data_raw['Neighborhood'].replace('NAmes', 'Names')
data_raw['BldgType'] = data_raw['BldgType'].replace({'2fmCon': '2FmCon', 'Duplex': 'Duplx','Twnhs': 'TwnhsE'})
data_raw['Exterior2nd'] = data_raw['Exterior2nd'].replace({'CmentBd':'CemntBd', 'Wd Sdng': 'WdSdng',  'Brk Cmn': 'BrkComm', 'Wd Shng': 'WdShing'})
data_raw['Exterior1st'] = data_raw['Exterior1st'].replace( 'Wd Sdng', 'WdSdng')
invalid_values = find_invalid_values(data_raw, valid_dict)
invalid_values


{'MSSubClass': [60,
  20,
  70,
  50,
  190,
  45,
  90,
  120,
  30,
  85,
  80,
  160,
  75,
  180,
  40,
  150],
 'OverallQual': [np.int64(7),
  np.int64(6),
  np.int64(8),
  np.int64(5),
  np.int64(9),
  np.int64(4),
  np.int64(10),
  np.int64(3),
  np.int64(1),
  np.int64(2)],
 'OverallCond': [np.int64(5),
  np.int64(8),
  np.int64(6),
  np.int64(7),
  np.int64(4),
  np.int64(2),
  np.int64(3),
  np.int64(9),
  np.int64(1)]}

## Missing Values

### Overview

In [21]:
def missing_values_table(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    missing_data = missing_data[missing_data['Total'] > 0]
    return missing_data

In [22]:
missing_df = missing_values_table(data_raw)

df1 = missing_df.iloc[:7, :]
df2 = missing_df[missing_df.index.str.contains('Garage')]
df3 = missing_df[missing_df.index.str.contains('Bsmt')]
df4 = missing_df.drop(df1.index.union(df2.index).union(df3.index))

display(HTML(f"""
<div style="display: flex; gap: 50px;">
    <div>{df1.to_html()}</div>
    <div>{df2.to_html()}</div>
    <div>{df3.to_html()}</div>
    <div>{df4.to_html()}</div>
</div>
"""))

,Total,Percent
PoolQC,2909,0.996574
MiscFeature,2814,0.964029
Alley,2721,0.932169
Fence,2348,0.804385
MasVnrType,1766,0.605002
FireplaceQu,1420,0.486468
LotFrontage,486,0.166495
,Total,Percent
GarageYrBlt,159,0.054471
GarageQual,159,0.054471


In [23]:
data_cleaned = data_raw.copy()

### Pool Variables

Features: Pool Quality, PoolArea

In [24]:
data_cleaned['PoolQC'].value_counts()

PoolQC
Ex    4
Gd    4
Fa    2
Name: count, dtype: int64

In [25]:
data_cleaned['PoolArea'].value_counts()

PoolArea
0      2906
512       1
648       1
576       1
555       1
480       1
519       1
738       1
144       1
368       1
444       1
228       1
561       1
800       1
Name: count, dtype: int64

In [26]:
data_cleaned = data_cleaned.drop(columns=["PoolQC", "PoolArea"])

### MiscFeature

In [27]:
data_cleaned['MiscFeature'].value_counts()

MiscFeature
Shed    95
Gar2     5
Othr     4
TenC     1
Name: count, dtype: int64

Shed can be refer to having garage/pool. Anyway, irrelevent feature.

In [28]:
data_cleaned.drop(columns=["MiscFeature"], inplace=True)

### Alley

In [29]:
data_cleaned['Alley'].value_counts()

Alley
Grvl    120
Pave     78
Name: count, dtype: int64

In [30]:
data_cleaned.drop(columns=["Alley"], inplace=True)

### Fence

In [31]:
data_cleaned['Fence'].value_counts()

Fence
MnPrv    329
GdPrv    118
GdWo     112
MnWw      12
Name: count, dtype: int64

In [32]:
data_cleaned.drop(columns=["Fence"], inplace=True)

### Fireplaces

Features: FireplaceQu, Fireplaces

FireplaceQu: Fireplace quality (1420 NA)

Ex   Excellent - Exceptional Masonry Fireplace  
Gd   Good - Masonry Fireplace in main level  
TA   Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement  
Fa   Fair - Prefabricated Fireplace in basement  
Po   Poor - Ben Franklin Stove  
NA   No Fireplace  

In [33]:
data_cleaned['FireplaceQu'].value_counts(dropna=False)

FireplaceQu
NaN    1420
Gd      744
TA      592
Fa       74
Po       46
Ex       43
Name: count, dtype: int64

In [34]:
from collections import defaultdict

In [35]:
quality_vector = defaultdict(int, {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1
})


In [36]:
data_cleaned['FireplaceQu'] = data_cleaned['FireplaceQu'].map(quality_vector)


In [37]:
data_cleaned['FireplaceQu'].value_counts(dropna=False)

FireplaceQu
0    1420
4     744
3     592
2      74
1      46
5      43
Name: count, dtype: int64

In [38]:
data_cleaned['Fireplaces'].value_counts(dropna=False) # maybe make it binary

Fireplaces
0    1420
1    1268
2     219
3      11
4       1
Name: count, dtype: int64

### Lot

Variables: LotFrontage (486 NA), LotArea, LotShape, LotConfig

LotFrontage: Linear feet of street connected to property => fill na with meadian per neigborhood, bc they rarely differ

In [39]:
median_vals = (
    data_cleaned.groupby("Neighborhood", as_index=False)["LotFrontage"].median()
)

fig = px.bar(
    median_vals,
    x="LotFrontage",
    y="Neighborhood",
    orientation="h",
    color_discrete_sequence=["blue"],
    title="Median LotFrontage by Neighborhood",
    text="LotFrontage" 
)

fig.update_traces(textposition="outside")

fig.update_layout(
    width=1200,
    height=600,
    yaxis=dict(categoryorder="total ascending"),
    xaxis_title="Median LotFrontage",
    yaxis_title="Neighborhood"
)

fig.show()

In [40]:
data_cleaned["LotFrontage"] = data_cleaned.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median())
)

Lot Area

In [41]:
data_cleaned["LotArea"]

0        8450
1        9600
2       11250
3        9550
4       14260
        ...  
2914     1936
2915     1894
2916    20000
2917    10441
2918     9627
Name: LotArea, Length: 2919, dtype: int64

LotConfig

In [42]:
data_cleaned["LotConfig"].value_counts(dropna=False)

LotConfig
Inside     2133
Corner      511
CulDSac     176
FR2          85
FR3          14
Name: count, dtype: int64

In [43]:
data_cleaned["LotConfig"] = data_cleaned["LotConfig"].astype("category")

LotShape

In [44]:
data_cleaned["LotShape"].value_counts(dropna=False)

LotShape
Reg    1859
IR1     968
IR2      76
IR3      16
Name: count, dtype: int64

Reg	Regular	  
IR1	Slightly irregular  
IR2	Moderately Irregular  
IR3	Irregular  

In [45]:
mapping = {'IR3': 0, 'IR2': 1, 'IR1': 2, 'Reg': 3}
data_cleaned['LotShape'] = data_cleaned['LotShape'].map(mapping).astype(int)


### Garage

GarageYrBlt	159	  
GarageQual	159	  
GarageCond	159	  
GarageFinish 159   
GarageType	157   
GarageArea	1  	    
GarageCars	1	 

Time of building Garage ~ time of building house

In [46]:
data_cleaned["GarageYrBlt"] = data_cleaned["GarageYrBlt"].fillna(data_cleaned["YearBuilt"])

Checking if all 157 NAs are the same observations among the variables with 157/159 NAs

In [47]:
mask = (
    data_cleaned["GarageType"].isna() &
    data_cleaned["GarageFinish"].isna() &
    data_cleaned["GarageCond"].isna() &
    data_cleaned["GarageQual"].isna()
)

count = mask.sum()
count

np.int64(157)

Checking other 2 NAs

In [48]:
cols = ['GarageCars', 'GarageArea', 'GarageType', 'GarageCond', 'GarageQual', 'GarageFinish']

subset = data_cleaned.loc[
    data_cleaned['GarageType'].notna() & data_cleaned['GarageFinish'].isna(),
    cols
]

subset

,GarageCars,GarageArea,GarageType,GarageCond,GarageQual,GarageFinish
2126,1.0,360.0,Detchd,NaN,NaN,NaN
2576,NaN,NaN,Detchd,NaN,NaN,NaN


Looks like 2126 has garage and 2576 not. 

In [49]:
cols = ['GarageCond', 'GarageQual', 'GarageFinish']

for col in cols:
    mode_val = data_cleaned[col].mode()[0]     
    data_cleaned.at[2126, col] = mode_val     

data_cleaned.at[2576, 'GarageCars'] = 0
data_cleaned.at[2576, 'GarageArea'] = 0
data_cleaned.at[2576, 'GarageType'] = np.nan
   


GarageType


2Types	       More than one type of garage     
Attchd	       Attached to home     
Basment	Basement Garage     
BuiltIn	Built-In (Garage part of house - typically has room above garage)       
CarPort	Car Port        
Detchd	       Detached from home       
NA	       No Garage        

In [50]:
data_cleaned['GarageType'] = data_cleaned['GarageType'].fillna('No Garage')
data_cleaned['GarageType'] = data_cleaned['GarageType'].astype('category')

GarageFinish

Fin  Finished       
RFn  Rough Finished     
Unf  Unfinished     
NA   No Garage          

In [51]:
data_cleaned['GarageFinish'] = data_cleaned['GarageFinish'].fillna('None')
finish_mapping = defaultdict(int, {
    'Fin': 3,
    'RFn': 2,
    'Unf': 1
})


In [52]:
data_cleaned['GarageFinish'] = data_cleaned['GarageFinish'].map(finish_mapping)

GarageQual, GarageCond

Ex   Excellent            
Gd   Good       
TA   Typical/Average        
Fa   Fair       
Po   Poor       
NA   No Garage      

In [53]:
data_cleaned['GarageQual'] = data_cleaned['GarageQual'].map(quality_vector)
data_cleaned['GarageCond'] = data_cleaned['GarageCond'].map(quality_vector)


### Basement

Features: BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath, BsmtHalfBath

In [54]:
df3

,Total,Percent
BsmtExposure,82,0.028092
BsmtCond,82,0.028092
BsmtQual,81,0.027749
BsmtFinType2,80,0.027407
BsmtFinType1,79,0.027064
BsmtFullBath,2,0.000685
BsmtHalfBath,2,0.000685
BsmtFinSF1,1,0.000343
BsmtFinSF2,1,0.000343
TotalBsmtSF,1,0.000343


Checking if all 79 NAs are the same observations among the variables with 80+ NAs

In [55]:
mask = (
    data_cleaned['BsmtQual'].isna() &
    data_cleaned['BsmtCond'].isna() &
    data_cleaned['BsmtExposure'].isna() &
    data_cleaned['BsmtFinType1'].isna() &
    data_cleaned['BsmtFinType2'].isna()
)

count = mask.sum()
count

np.int64(79)

Checking others NAs

In [56]:
cols = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']

subset = data_cleaned.loc[
    data_cleaned['BsmtFinType1'].notna() & 
    (data_cleaned['BsmtCond'].isna() | data_cleaned['BsmtQual'].isna() | data_cleaned['BsmtExposure'].isna() | data_cleaned['BsmtFinType2'].isna()),
    cols
]

subset

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2
332,Gd,TA,No,GLQ,NaN
948,Gd,TA,NaN,Unf,Unf
1487,Gd,TA,NaN,Unf,Unf
2040,Gd,NaN,Mn,GLQ,Rec
2185,TA,NaN,No,BLQ,Unf
2217,NaN,Fa,No,Unf,Unf
2218,NaN,TA,No,Unf,Unf
2348,Gd,TA,NaN,Unf,Unf
2524,TA,NaN,Av,ALQ,Unf


In [57]:
cols_indices = {
    'BsmtFinType2': [333],
    'BsmtExposure': [949, 1488, 2349],
    'BsmtCond': [2041, 2186, 2525],
    'BsmtQual': [2218, 2219]
}

for col, idxs in cols_indices.items():
    mode_val = data_cleaned[col].mode()[0]       
    data_cleaned.loc[idxs, col] = mode_val      

BsmtQual

Ex   Excellent (100+ inches)    
Gd   Good (90-99 inches)        
TA   Typical (80-89 inches)         
Fa   Fair (70-79 inches)        
Po   Poor (<70 inches)       
NA   No Basement        

In [58]:
data_cleaned['BsmtQual'] = data_cleaned['BsmtQual'].map(quality_vector)

In [59]:
data_cleaned['BsmtQual'].value_counts(dropna=False)

BsmtQual
3    1285
4    1209
5     258
2      87
0      80
Name: count, dtype: int64

BsmtCond

Ex   Excellent      
Gd   Good       
TA   Typical - slight dampness allowed      
Fa   Fair - dampness or some cracking or settling       
Po   Poor - Severe cracking, settling, or wetness       
NA   No Basement        

In [60]:
data_cleaned['BsmtCond'] = data_cleaned['BsmtCond'].map(quality_vector)
data_cleaned['BsmtCond'].value_counts(dropna=False)

BsmtCond
3    2606
4     122
2     104
0      82
1       5
Name: count, dtype: int64

BsmtExposure

Gd   Good Exposure      
Av   Average Exposure (split levels or foyers typically score average or above)     
Mn   Mimimum Exposure       
No   No Exposure        
NA   No Basement        

In [61]:
exposure_vector = defaultdict(int, {'No':1, 'Mn':2, 'Av':3, 'Gd':4})

In [62]:
data_cleaned['BsmtExposure'] = data_cleaned['BsmtExposure'].map(exposure_vector)
data_cleaned['BsmtExposure'].value_counts(dropna=False)

BsmtExposure
1    1905
3     417
4     276
2     239
0      82
Name: count, dtype: int64

BsmtFinType1, BsmtFinType2

GLQ  Good Living Quarters       
ALQ  Average Living Quarters        
BLQ  Below Average Living Quarters          
Rec  Average Rec Room       
LwQ  Low Quality        
Unf  Unfinshed      
NA   No Basement        

In [63]:
FinType = defaultdict(int,  {'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6})
data_cleaned['BsmtFinType1'] = data_cleaned['BsmtFinType1'].map(FinType)
data_cleaned['BsmtFinType1'].value_counts(dropna=False)

BsmtFinType1
1    851
6    849
5    429
3    288
4    269
2    154
0     79
Name: count, dtype: int64

In [64]:
data_cleaned['BsmtFinType2'] = data_cleaned['BsmtFinType2'].map(FinType)
data_cleaned['BsmtFinType2'].value_counts(dropna=False)

BsmtFinType2
1    2493
3     105
2      87
0      80
4      68
5      52
6      34
Name: count, dtype: int64

Dealing with 1-2 NAs

In [65]:
cols = ['BsmtQual', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF']

mask = (
    data_cleaned['BsmtFullBath'].isna() |
    data_cleaned['BsmtHalfBath'].isna() |
    data_cleaned['BsmtFinSF1'].isna() |
    data_cleaned['BsmtFinSF2'].isna() |
    data_cleaned['BsmtUnfSF'].isna() |
    data_cleaned['TotalBsmtSF'].isna()
)

subset = data_cleaned.loc[mask, cols]
subset

,BsmtQual,BsmtFullBath,BsmtHalfBath,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
2120,0,NaN,NaN,NaN,NaN,NaN,NaN
2188,0,NaN,NaN,0.0,0.0,0.0,0.0


looks like that those remaining NAs all refer to "not present"

In [66]:
data_cleaned[cols[1:]] = data_cleaned[cols[1:]].fillna(0)

### Masonry 

Features: MasVnrArea(23 NA), MasVnrType(1766 NA)

In [67]:
data_cleaned['MasVnrType'].value_counts(dropna=False)

MasVnrType
NaN        1766
BrkFace     879
Stone       249
BrkCmn       25
Name: count, dtype: int64

In [68]:
mask_area_na = data_cleaned['MasVnrArea'].isna()
len(data_cleaned.loc[mask_area_na, ['MasVnrArea', 'MasVnrType']])

23

In [69]:
data_cleaned.drop(columns=['MasVnrType'], inplace=True)
data_cleaned['MasVnrArea'] = data_cleaned['MasVnrArea'].fillna(0)

### MSZoning


A    Agriculture        
C    Commercial     
FV   Floating Village Residential       
I    Industrial     
RH   Residential High Density       
RL   Residential Low Density        
RP   Residential Low Density Park       
RM   Residential Medium Density     

In [70]:
data_cleaned['MSZoning'] = data_cleaned['MSZoning'].fillna(data_cleaned['MSZoning'].mode()[0])
data_cleaned['MSZoning'] = data_cleaned['MSZoning'].astype('category')


### One/Two NAs

In [71]:
missing_cleaned = missing_values_table(data_cleaned)
missing_cleaned
print("Features with missing values:")
print(missing_cleaned)

Features with missing values:
             Total   Percent
Utilities        2  0.000685
Functional       2  0.000685
Exterior1st      1  0.000343
Electrical       1  0.000343
SaleType         1  0.000343
KitchenQual      1  0.000343
Exterior2nd      1  0.000343


In [72]:
data_cleaned['KitchenQual'] = data_cleaned['KitchenQual'].map(quality_vector)
data_cleaned['KitchenQual'].value_counts(dropna=False)

KitchenQual
3    1492
4    1151
5     205
2      70
0       1
Name: count, dtype: int64

In [73]:
data_cleaned['SaleType'] = data_cleaned['SaleType'].fillna(data_cleaned['SaleType'].mode()[0])
data_cleaned['SaleType'] = data_cleaned['SaleType'].astype('category')
data_cleaned['SaleType'].value_counts(dropna=False)

SaleType
WD       2526
New       239
COD        87
ConLD      26
CWD        12
ConLI       9
ConLw       8
Oth         7
Con         5
Name: count, dtype: int64

In [74]:
print(data_cleaned['Utilities'].value_counts(dropna=False))
data_cleaned.drop(columns=['Utilities'], inplace=True)


Utilities
AllPub    2916
NaN          2
NoSeWa       1
Name: count, dtype: int64


In [75]:
print(data_cleaned['Functional'].value_counts(dropna=False))

Functional
Typ     2717
Min2      70
Min1      65
Mod       35
Maj1      19
Maj2       9
Sev        2
NaN        2
Name: count, dtype: int64


Typ	Typical Functionality       
Min1	Minor Deductions 1      
Min2	Minor Deductions 2      
Mod	Moderate Deductions     
Maj1	Major Deductions 1          
Maj2	Major Deductions 2      
Sev	Severely Damaged        
Sal	Salvage only        

In [76]:
functional_map  = defaultdict(int,  {'Sal': 0,
    'Sev': 1,
    'Maj2': 2,
    'Maj1': 3,
    'Mod': 4,
    'Min2': 5,
    'Min1': 6,
    'Typ': 7
})
mode_value = data_cleaned['Functional'].mode()[0]
data_cleaned['Functional'] = data_cleaned['Functional'].fillna(mode_value)
data_cleaned['Functional'] = data_cleaned['Functional'].map(functional_map).astype(int)
print(data_cleaned['Functional'].value_counts())

Functional
7    2719
5      70
6      65
4      35
3      19
2       9
1       2
Name: count, dtype: int64


In [77]:
data_cleaned['Exterior1st'] = data_cleaned['Exterior1st'].fillna(data_cleaned['Exterior1st'].mode()[0])
data_cleaned['Exterior1st'] = data_cleaned['Exterior1st'].astype('category')
data_cleaned['Exterior1st'].value_counts(dropna=False)

Exterior1st
VinylSd    1026
MetalSd     450
HdBoard     442
WdSdng      411
Plywood     221
CemntBd     126
BrkFace      87
WdShing      56
AsbShng      44
Stucco       43
BrkComm       6
CBlock        2
AsphShn       2
Stone         2
ImStucc       1
Name: count, dtype: int64

In [78]:
data_cleaned['Exterior2nd'] = data_cleaned['Exterior2nd'].fillna(data_cleaned['Exterior2nd'].mode()[0])
data_cleaned['Exterior2nd'] = data_cleaned['Exterior2nd'].astype('category')
data_cleaned['Exterior2nd'].value_counts(dropna=False)

Exterior2nd
VinylSd    1015
MetalSd     447
HdBoard     406
WdSdng      391
Plywood     270
CemntBd     126
WdShing      81
BrkFace      47
Stucco       47
AsbShng      38
BrkComm      22
ImStucc      15
Stone         6
AsphShn       4
CBlock        3
Other         1
Name: count, dtype: int64

In [79]:
data_cleaned['Electrical'] = data_cleaned['Electrical'].fillna(data_cleaned['Electrical'].mode()[0])
data_cleaned['Electrical'] = data_cleaned['Electrical'].astype('category')
data_cleaned['Electrical'].value_counts(dropna=False)

Electrical
SBrkr    2672
FuseA     188
FuseF      50
FuseP       8
Mix         1
Name: count, dtype: int64

In [80]:
missing_values_table(data_cleaned)

,Total,Percent


## Categorizing

In [81]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 73 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   MSSubClass     2919 non-null   category
 1   MSZoning       2919 non-null   category
 2   LotFrontage    2919 non-null   float64 
 3   LotArea        2919 non-null   int64   
 4   Street         2919 non-null   object  
 5   LotShape       2919 non-null   int64   
 6   LandContour    2919 non-null   object  
 7   LotConfig      2919 non-null   category
 8   LandSlope      2919 non-null   object  
 9   Neighborhood   2919 non-null   object  
 10  Condition1     2919 non-null   object  
 11  Condition2     2919 non-null   object  
 12  BldgType       2919 non-null   object  
 13  HouseStyle     2919 non-null   object  
 14  OverallQual    2919 non-null   int64   
 15  OverallCond    2919 non-null   int64   
 16  YearBuilt      2919 non-null   int64   
 17  YearRemodAdd   2919 non-null   in

In [82]:
data_cleaned['Neighborhood'] = data_cleaned['Neighborhood'].astype('category')

In [83]:
obj_cols = data_cleaned.select_dtypes(include='object').columns
tables_html = []
for col in obj_cols:
    vc = data_cleaned[col].value_counts(dropna=False).to_frame(name="count")
    vc.index.name = col
    tables_html.append(vc.to_html())

html_output = '<div style="display: flex; flex-wrap: wrap; gap: 40px;">'
for tbl in tables_html:
    html_output += f"<div>{tbl}</div>"
html_output += "</div>"

display(HTML(html_output))


,count
Street,
Pave,2907
Grvl,12
,count
LandContour,
Lvl,2622
HLS,120
Bnk,117
Low,60
,count


In [84]:
data_cleaned.drop(columns=['Street', 'Heating', "RoofMatl"], inplace=True)

In [85]:
data_cleaned['ExterQual'] = data_cleaned['ExterQual'].map(quality_vector)
data_cleaned['ExterCond'] = data_cleaned['ExterCond'].map(quality_vector)
data_cleaned['HeatingQC'] = data_cleaned['HeatingQC'].map(quality_vector)


In [86]:
data_cleaned['Foundation'] = data_cleaned['Foundation'].astype('category')
data_cleaned['RoofStyle'] = data_cleaned['RoofStyle'].astype('category')
data_cleaned['LandContour'] = data_cleaned['LandContour'].astype('category')
data_cleaned['BldgType'] = data_cleaned['BldgType'].astype('category')
data_cleaned['HouseStyle'] = data_cleaned['HouseStyle'].astype('category')
data_cleaned['Condition1'] = data_cleaned['Condition1'].astype('category')
data_cleaned['Condition2'] = data_cleaned['Condition2'].astype('category')
data_cleaned['SaleCondition'] = data_cleaned['SaleCondition'].astype('category')

In [87]:
slope_map = defaultdict(int, {'Gtl': 2, 'Mod': 1, 'Sev': 0})    
data_cleaned['LandSlope'] = data_cleaned['LandSlope'].map(slope_map).astype(int)

In [88]:
paved_map = defaultdict(int, {'Y': 2, 'P': 1, 'N': 0})    
data_cleaned['PavedDrive'] = data_cleaned['PavedDrive'].map(paved_map).astype(int)

In [89]:
data_cleaned['CentralAir'] = data_cleaned['CentralAir'].map({'Y': 1, 'N': 0}).astype(int)

In [90]:
print(data_cleaned['MoSold'].value_counts(dropna=False))
print(data_cleaned['YrSold'].value_counts(dropna=False))

MoSold
6     503
7     446
5     394
4     279
8     233
3     232
10    173
9     158
11    142
2     133
1     122
12    104
Name: count, dtype: int64
YrSold
2007    692
2009    647
2008    622
2006    619
2010    339
Name: count, dtype: int64


In [91]:
data_cleaned['MoSold'] = data_cleaned['MoSold'].astype('category')

In [92]:
data_cleaned['ExterQual'].value_counts(dropna=False)

ExterQual
3    1798
4     979
5     107
2      35
Name: count, dtype: int64

## Feature Engineering

### Reduce count of features about Bathrooms

In [93]:
data_cleaned['TotBathrooms'] = (
    data_cleaned['FullBath'] +
    data_cleaned['HalfBath'] * 0.5 +
    data_cleaned['BsmtFullBath'] +
    data_cleaned['BsmtHalfBath'] * 0.5
)
data_cleaned.drop(columns=['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath'], inplace=True)

In [94]:
corr_value = data_cleaned['TotBathrooms'].corr(data_cleaned['SalePrice'])

fig = px.scatter(
    data_cleaned, 
    x='TotBathrooms', 
    y='SalePrice', 
    trendline='ols',
    title=f'SalePrice vs TotBathrooms (corr={corr_value:.2f})'
)

fig.update_layout(
    xaxis_title='Total Bathrooms',
    yaxis_title='SalePrice'
)
fig.show()

### Adding House Age, IsNew (Yes/No)

In [95]:
data_cleaned['Age'] = data_cleaned['YrSold'] - data_cleaned['YearRemodAdd']


In [96]:
cols = ['YearRemodAdd', 'YrSold', 'Age']

fig = make_subplots(rows=1, cols=3, subplot_titles=cols)

for i, col in enumerate(cols, start=1):
    corr = data_cleaned[col].corr(data_cleaned['SalePrice'])
    
    X = sm.add_constant(data_cleaned[col])
    model = sm.OLS(data_cleaned['SalePrice'], X).fit()
    trend = model.predict(X)
    
    fig.add_trace(
        go.Scatter(
            x=data_cleaned[col],
            y=data_cleaned['SalePrice'],
            mode='markers',
            name=col,
            hovertext=data_cleaned.index,
            hovertemplate='ID: %{hovertext}<br>'+col+': %{x}<br>SalePrice: %{y}<extra></extra>'
        ),
        row=1, col=i
    )
    
    fig.add_trace(
        go.Scatter(
            x=data_cleaned[col],
            y=trend,
            mode='lines',
            line=dict(color='red'),
            name='Trend',
            showlegend=False
        ),
        row=1, col=i
    )
    fig.layout.annotations[i-1].text = f"{col} (corr={corr:.2f})"

fig.update_layout(width=1800, height=500, showlegend=False)
fig.show()

In [97]:
data_cleaned['IsNew'] = np.where(
    data_cleaned['YrSold'] == data_cleaned['YearBuilt'], 0, 1
)

In [98]:
grouped = data_cleaned.groupby('IsNew').agg(
    MedianSalePrice=('SalePrice', 'median'),
    Count=('SalePrice', 'count')
).reset_index()

fig = px.bar(
    grouped, 
    x='IsNew', 
    y='MedianSalePrice', 
    text='Count', 
    color_discrete_sequence=['blue']
)

fig.update_layout(
    title="Median SalePrice by IsNew",
    xaxis_title="IsNew (0=No, 1=Yes)",
    yaxis_title="SalePrice",
    yaxis=dict(tick0=0, dtick=50000),
    template="plotly_white"
)

fig.show()

In [99]:
data_cleaned.drop(columns=['YearRemodAdd', 'YrSold'], inplace=True)

### Binning Neighborhood

In [100]:
median_grouped = data_cleaned.groupby('Neighborhood').agg(
    MedianSalePrice=('SalePrice', 'median'),
    Count=('SalePrice', 'count')
).reset_index()

mean_grouped = data_cleaned.groupby('Neighborhood').agg(
    MeanSalePrice=('SalePrice', 'mean'),
    Count=('SalePrice', 'count')
).reset_index()

overall_median = data_cleaned['SalePrice'].median()
overall_mean = data_cleaned['SalePrice'].mean()

fig = make_subplots(rows=1, cols=2, subplot_titles=("Median SalePrice by Neighborhood", "Mean SalePrice by Neighborhood"))

fig.add_trace(
    go.Bar(
        x=median_grouped.sort_values('MedianSalePrice')['Neighborhood'],
        y=median_grouped.sort_values('MedianSalePrice')['MedianSalePrice'],
        text=median_grouped.sort_values('MedianSalePrice')['Count'],
        textposition='auto',
        marker_color='blue',
        name='Median'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=mean_grouped.sort_values('MeanSalePrice')['Neighborhood'],
        y=mean_grouped.sort_values('MeanSalePrice')['MeanSalePrice'],
        text=mean_grouped.sort_values('MeanSalePrice')['Count'],
        textposition='auto',
        marker_color='blue',
        name='Mean'
    ),
    row=1, col=2
)

fig.add_hline(y=overall_median, line_dash="dash", line_color="red", row=1, col=1)
fig.add_hline(y=overall_mean, line_dash="dash", line_color="red", row=1, col=2)

fig.update_layout(
    width=1800,
    height=400,
    showlegend=False,
    xaxis_tickangle=-45,
    xaxis2_tickangle=-45
)

fig.show()

C:\Users\grund\AppData\Local\Temp\ipykernel_10524\2896675624.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\grund\AppData\Local\Temp\ipykernel_10524\2896675624.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [101]:
median_dict = median_grouped.set_index('Neighborhood')['MedianSalePrice'].to_dict()
def assign_neigh_rich(neighborhood):
    median_value = median_dict.get(neighborhood, 0)
    if median_value < 150000:
        return 0
    elif median_value < 180000:
        return 1
    elif median_value < 215000:
        return 2
    else:
        return 3

data_cleaned['NeighRich'] = data_cleaned['Neighborhood'].apply(assign_neigh_rich)


In [102]:
data_cleaned.drop(columns=['Neighborhood'], inplace=True)

### Total Square Feet

In [103]:
data_cleaned["TotalSqFeet"] = data_cleaned['GrLivArea'] + data_cleaned["TotalBsmtSF"]

In [104]:
cols = ['GrLivArea', 'TotalBsmtSF', 'TotalSqFeet']

fig = make_subplots(rows=1, cols=3, subplot_titles=cols)

for i, col in enumerate(cols, start=1):

    corr = data_cleaned[col].corr(data_cleaned['SalePrice'])

    X = sm.add_constant(data_cleaned[col])
    model = sm.OLS(data_cleaned['SalePrice'], X).fit()
    trend = model.predict(X)
    

    fig.add_trace(
        go.Scatter(
            x=data_cleaned[col],
            y=data_cleaned['SalePrice'],
            mode='markers',
            name=col,
            hovertext=data_cleaned.index, 
            hovertemplate='ID: %{hovertext}<br>'+col+': %{x}<br>SalePrice: %{y}<extra></extra>'
        ),
        row=1, col=i
    )

    fig.add_trace(
        go.Scatter(
            x=data_cleaned[col],
            y=trend,
            mode='lines',
            line=dict(color='red'),
            name='Trend'
        ),
        row=1, col=i
    )
    
    fig.layout.annotations[i-1].text = f"{col} (corr={corr:.2f})"

fig.update_layout(width=1800, height=500, showlegend=False)
fig.show()

As we see corr increased. Also we can delete outliers with ids: 523, 2549, 1298

In [105]:
data_cleaned = data_cleaned.drop(index=[523, 2549, 1298])

In [106]:
cols = ['GrLivArea', 'TotalBsmtSF', 'TotalSqFeet']

fig = make_subplots(rows=1, cols=3, subplot_titles=cols)

for i, col in enumerate(cols, start=1):

    corr = data_cleaned[col].corr(data_cleaned['SalePrice'])

    X = sm.add_constant(data_cleaned[col])
    model = sm.OLS(data_cleaned['SalePrice'], X).fit()
    trend = model.predict(X)
    

    fig.add_trace(
        go.Scatter(
            x=data_cleaned[col],
            y=data_cleaned['SalePrice'],
            mode='markers',
            name=col,
            hovertext=data_cleaned.index, 
            hovertemplate='ID: %{hovertext}<br>'+col+': %{x}<br>SalePrice: %{y}<extra></extra>'
        ),
        row=1, col=i
    )

    fig.add_trace(
        go.Scatter(
            x=data_cleaned[col],
            y=trend,
            mode='lines',
            line=dict(color='red'),
            name='Trend'
        ),
        row=1, col=i
    )
    
    fig.layout.annotations[i-1].text = f"{col} (corr={corr:.2f})"

fig.update_layout(width=1800, height=500, showlegend=False)
fig.show()

### LowQualFinSF

In [107]:
data_cleaned['LowQualFinSF'].value_counts()

LowQualFinSF
0       2876
80         4
205        2
360        2
234        1
513        1
144        1
528        1
371        1
390        1
420        1
572        1
473        1
156        1
515        1
53         1
232        1
481        1
120        1
392        1
514        1
397        1
479        1
384        1
362        1
1064       1
431        1
436        1
259        1
312        1
108        1
697        1
512        1
114        1
140        1
450        1
Name: count, dtype: int64

In [108]:
data_cleaned.drop(columns=['LowQualFinSF'], inplace=True)

In [109]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2916 entries, 0 to 2918
Data columns (total 67 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   MSSubClass     2916 non-null   category
 1   MSZoning       2916 non-null   category
 2   LotFrontage    2916 non-null   float64 
 3   LotArea        2916 non-null   int64   
 4   LotShape       2916 non-null   int64   
 5   LandContour    2916 non-null   category
 6   LotConfig      2916 non-null   category
 7   LandSlope      2916 non-null   int64   
 8   Condition1     2916 non-null   category
 9   Condition2     2916 non-null   category
 10  BldgType       2916 non-null   category
 11  HouseStyle     2916 non-null   category
 12  OverallQual    2916 non-null   int64   
 13  OverallCond    2916 non-null   int64   
 14  YearBuilt      2916 non-null   int64   
 15  RoofStyle      2916 non-null   category
 16  Exterior1st    2916 non-null   category
 17  Exterior2nd    2916 non-null   categor

### Porches

In [110]:
data_cleaned['TotalPorchSF'] = (
    data_cleaned['OpenPorchSF'] +
    data_cleaned['EnclosedPorch'] +
    data_cleaned['3SsnPorch'] +
    data_cleaned['ScreenPorch']
)


In [111]:
cols = ['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'TotalPorchSF']

fig = make_subplots(rows=2, cols=3, subplot_titles=cols)

for i, col in enumerate(cols):
    # Кореляція
    corr = data_cleaned[col].corr(data_cleaned['SalePrice'])
    
    # Лінія тренду
    X = sm.add_constant(data_cleaned[col])
    model = sm.OLS(data_cleaned['SalePrice'], X).fit()
    trend = model.predict(X)
    
    # Обчислюємо ряд і колонку для subplot
    row = i // 3 + 1
    col_num = i % 3 + 1
    
    # Точки з hovertext
    fig.add_trace(
        go.Scatter(
            x=data_cleaned[col],
            y=data_cleaned['SalePrice'],
            mode='markers',
            hovertext=data_cleaned.index,
            hovertemplate='ID: %{hovertext}<br>'+col+': %{x}<br>SalePrice: %{y}<extra></extra>'
        ),
        row=row, col=col_num
    )
    
    # Лінія тренду
    fig.add_trace(
        go.Scatter(
            x=data_cleaned[col],
            y=trend,
            mode='lines',
            line=dict(color='red'),
            showlegend=False
        ),
        row=row, col=col_num
    )
    
    # Оновлюємо заголовок
    fig.layout.annotations[i].text = f"{col} (corr={corr:.2f})"

fig.update_layout(width=1800, height=800, showlegend=False)
fig.show()

In [112]:
data_cleaned.drop(columns=['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'], inplace=True)

In [113]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2916 entries, 0 to 2918
Data columns (total 64 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   MSSubClass     2916 non-null   category
 1   MSZoning       2916 non-null   category
 2   LotFrontage    2916 non-null   float64 
 3   LotArea        2916 non-null   int64   
 4   LotShape       2916 non-null   int64   
 5   LandContour    2916 non-null   category
 6   LotConfig      2916 non-null   category
 7   LandSlope      2916 non-null   int64   
 8   Condition1     2916 non-null   category
 9   Condition2     2916 non-null   category
 10  BldgType       2916 non-null   category
 11  HouseStyle     2916 non-null   category
 12  OverallQual    2916 non-null   int64   
 13  OverallCond    2916 non-null   int64   
 14  YearBuilt      2916 non-null   int64   
 15  RoofStyle      2916 non-null   category
 16  Exterior1st    2916 non-null   category
 17  Exterior2nd    2916 non-null   categor

## Correlations

Identifying the most important features for predicting our target variable - SalePrice

In [114]:
num_vars = data_cleaned.select_dtypes(include=['number']) 
corr = num_vars.corr(method='pearson') 
cor_sorted = corr['SalePrice'].sort_values(ascending=True)
high_corr = cor_sorted[abs(cor_sorted) > 0.5].index
corr_high = corr.loc[high_corr, high_corr]

fig = px.imshow(
    corr_high.values,
    x=corr_high.columns,
    y=corr_high.index,
    color_continuous_scale='RdBu_r',
    origin='lower',
    text_auto=".2f",
    aspect="auto"
)

fig.update_layout(
    title="Correlation Matrix (|corr| > 0.5)",
    xaxis_title="Features",
    yaxis_title="Features",
    width=800,
    height=800
)

fig.show()


In [115]:
num_vars = data_cleaned.select_dtypes(include=['number'])

corr_matrix = num_vars.corr()  
abs_corr_matrix = corr_matrix.abs()  

upper = abs_corr_matrix.where(np.triu(np.ones(abs_corr_matrix.shape), k=1).astype(bool))

high_corr = (
    upper.stack()
    .reset_index()
    .rename(columns={'level_0': 'Feature1', 'level_1': 'Feature2', 0: 'Correlation'})
)

high_corr = high_corr[high_corr['Correlation'] > 0.65]

high_corr['Corr_with_SalePrice_Feature1'] = high_corr['Feature1'].map(corr_matrix['SalePrice'])
high_corr['Corr_with_SalePrice_Feature2'] = high_corr['Feature2'].map(corr_matrix['SalePrice'])

high_corr = high_corr.sort_values(by='Correlation', ascending=False).reset_index(drop=True)

high_corr

,Feature1,Feature2,Correlation,Corr_with_SalePrice_Feature1,Corr_with_SalePrice_Feature2
0,GarageQual,GarageCond,0.946370,0.197883,0.188374
1,GarageCars,GarageArea,0.892470,0.468863,0.466690
2,GrLivArea,TotalSqFeet,0.864975,0.603196,0.645310
3,Fireplaces,FireplaceQu,0.862455,0.354544,0.386067
4,YearBuilt,GarageYrBlt,0.852807,0.362136,0.342428
5,TotalBsmtSF,TotalSqFeet,0.811514,0.471256,0.645310
6,GrLivArea,TotRmsAbvGrd,0.807787,0.603196,0.472680
7,BsmtFinType2,BsmtFinSF2,0.799281,-0.009763,-0.007337
8,TotalBsmtSF,1stFlrSF,0.786260,0.471256,0.478124
9,1stFlrSF,TotalSqFeet,0.775110,0.478124,0.645310


In [116]:

data_filtered = data_cleaned.copy()

while True:
    num_vars = data_filtered.select_dtypes(include=['number'])
    corr_matrix = num_vars.corr()
    abs_corr_matrix = corr_matrix.abs()

    upper = abs_corr_matrix.where(np.triu(np.ones(abs_corr_matrix.shape), k=1).astype(bool))

    high_corr = (
        upper.stack()
        .reset_index()
        .rename(columns={'level_0': 'Feature1', 'level_1': 'Feature2', 0: 'Correlation'})
    )

    high_corr = high_corr[high_corr['Correlation'] > 0.7]
    
    if high_corr.empty:
        break 
    
    high_corr['Corr_with_SalePrice_Feature1'] = high_corr['Feature1'].map(corr_matrix['SalePrice'])
    high_corr['Corr_with_SalePrice_Feature2'] = high_corr['Feature2'].map(corr_matrix['SalePrice'])
    

    high_corr = high_corr.sort_values(by='Correlation', ascending=False).reset_index(drop=True)
    
    row = high_corr.iloc[0]
    
    if abs(row['Corr_with_SalePrice_Feature1']) < abs(row['Corr_with_SalePrice_Feature2']):
        drop_feature = row['Feature1']
    else:
        drop_feature = row['Feature2']
    
    print(f"Dropping feature: {drop_feature} (correlation with SalePrice={corr_matrix['SalePrice'][drop_feature]:.3f})")

    data_filtered = data_filtered.drop(columns=[drop_feature])

Dropping feature: GarageCond (correlation with SalePrice=0.188)
Dropping feature: GarageArea (correlation with SalePrice=0.467)
Dropping feature: GrLivArea (correlation with SalePrice=0.603)
Dropping feature: Fireplaces (correlation with SalePrice=0.355)
Dropping feature: GarageYrBlt (correlation with SalePrice=0.342)
Dropping feature: TotalBsmtSF (correlation with SalePrice=0.471)
Dropping feature: BsmtFinSF2 (correlation with SalePrice=-0.007)
Dropping feature: 1stFlrSF (correlation with SalePrice=0.478)
Dropping feature: ExterQual (correlation with SalePrice=0.469)
Dropping feature: BsmtFinType1 (correlation with SalePrice=0.199)


In [117]:
data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2916 entries, 0 to 2918
Data columns (total 54 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   MSSubClass     2916 non-null   category
 1   MSZoning       2916 non-null   category
 2   LotFrontage    2916 non-null   float64 
 3   LotArea        2916 non-null   int64   
 4   LotShape       2916 non-null   int64   
 5   LandContour    2916 non-null   category
 6   LotConfig      2916 non-null   category
 7   LandSlope      2916 non-null   int64   
 8   Condition1     2916 non-null   category
 9   Condition2     2916 non-null   category
 10  BldgType       2916 non-null   category
 11  HouseStyle     2916 non-null   category
 12  OverallQual    2916 non-null   int64   
 13  OverallCond    2916 non-null   int64   
 14  YearBuilt      2916 non-null   int64   
 15  RoofStyle      2916 non-null   category
 16  Exterior1st    2916 non-null   category
 17  Exterior2nd    2916 non-null   categor

# Analysis of feature to predict

In [118]:
fig = px.histogram(
    data_filtered,
    x="SalePrice",
    nbins=100,
    marginal="box",
    histnorm="probability density",
    opacity=0.7
)

# KDE через statsmodels
kde = sm.nonparametric.KDEUnivariate(data_filtered["SalePrice"])
kde.fit(bw="scott")  # можна "scott", "silverman" або число

# додаємо KDE криву
fig.add_scatter(
    x=kde.support,
    y=kde.density,
    mode="lines",
    line=dict(color="red"),
    name="KDE"
)

fig.update_layout(
    title="SalePrice distribution",
    xaxis_title="SalePrice",
    yaxis_title="Density",
    bargap=0.05,
    showlegend=True,
    width=900,
    height=600
)

fig.show()

In [119]:
print(f"Skewness: {data_filtered['SalePrice'].skew():.4f}") # наскільки розподіл "косий" відносно нормального
print(f"Kurtosis: {data_filtered['SalePrice'].kurt():.4f}") # наскільки "гострий" або "плоский" розподіл у порівнянні з нормальним


Skewness: 2.5495
Kurtosis: 14.6599


In [120]:
data_filtered["SalePriceLog"] = np.log(data_filtered["SalePrice"])

In [121]:
mu, sigma = norm.fit(data_filtered['SalePriceLog'])
print(f"\n mu = {mu:.2f} and sigma = {sigma:.2f}\n")

# гістограма з KDE
fig = px.histogram(
    data_filtered,
    x="SalePriceLog",
    nbins=100,
    histnorm="probability density",
    opacity=0.6,
    marginal="box",   # опціонально
    title="SalePrice distribution"
)

x = np.linspace(data_filtered["SalePriceLog"].min(), data_filtered["SalePriceLog"].max(), 200)
p = norm.pdf(x, mu, sigma)

fig.add_trace(
    go.Scatter(
        x=x,
        y=p,
        mode="lines",
        line=dict(color="red", width=2),
        name=f"Normal dist. (μ={mu:.2f}, σ={sigma:.2f})"
    )
)

fig.update_layout(
    xaxis_title="SalePrice (log)",
    yaxis_title="Density",
    width=850,
    height=600
)

fig.show()


 mu = 12.06 and sigma = 0.29



Спостерігається гарний збіг у центральній частині розподілу (близько μ ~ 12.06),   
що є основною ознакою успішного перетворення.  
Емпіричний розподіл має більш важкі, ніж у теоретичної норми, хвости з обох сторін  (в інтервалах ~11.0-11.5 зліва та ~12.5-13.0 праворуч).  
Це означає, що у даних є більше екстремальних значень (як дуже низьких, і дуже високих)

In [122]:
# data_cleaned.drop(columns=["SalePrice"], inplace=True)

# FE2 & preprocessing predictor variables

In [123]:
data_predictors = data_filtered.drop(columns=["SalePrice", "SalePriceLog"])
data_target = data_filtered["SalePriceLog"]

In [124]:
data_predictors.select_dtypes(include=np.number)


,LotFrontage,LotArea,LotShape,LandSlope,OverallQual,OverallCond,YearBuilt,MasVnrArea,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinType2,BsmtUnfSF,HeatingQC,CentralAir,2ndFlrSF,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,FireplaceQu,GarageFinish,GarageCars,GarageQual,PavedDrive,WoodDeckSF,MiscVal,TotBathrooms,Age,IsNew,NeighRich,TotalSqFeet,TotalPorchSF
0,65.0,8450,3,2,7,5,2003,196.0,3,4,3,1,706.0,1,150.0,5,1,854,3,1,4,8,7,0,2,2.0,3,2,0,0,3.5,5,1,2,2566.0,61
1,80.0,9600,3,2,6,8,1976,0.0,3,4,3,4,978.0,1,284.0,5,1,0,3,1,3,6,7,3,2,2.0,3,2,298,0,2.5,31,1,2,2524.0,0
2,68.0,11250,2,2,7,5,2001,162.0,3,4,3,2,486.0,1,434.0,5,1,866,3,1,4,6,7,3,2,2.0,3,2,0,0,3.5,6,1,2,2706.0,42
3,60.0,9550,2,2,7,5,1915,0.0,3,3,4,1,216.0,1,540.0,4,1,756,3,1,4,7,7,4,1,3.0,3,2,0,0,2.0,36,1,2,2473.0,307
4,84.0,14260,2,2,8,5,2000,350.0,3,4,3,3,655.0,1,490.0,5,1,1053,4,1,4,9,7,3,2,3.0,3,2,192,0,3.5,8,1,3,3343.0,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,21.0,1936,3,2,4,7,1970,0.0,3,3,3,1,0.0,1,546.0,4,1,546,3,1,3,5,7,0,0,0.0,0,2,0,0,1.5,36,1,1,1638.0,0
2915,21.0,1894,3,2,4,5,1970,0.0,3,3,3,1,252.0,1,294.0,3,1,546,3,1,3,6,7,0,1,1.0,3,2,0,0,1.5,36,1,1,1638.0,24
2916,160.0,20000,3,2,5,7,1960,0.0,3,3,3,1,1224.0,1,0.0,5,1,0,4,1,3,7,7,3,1,2.0,3,2,474,0,2.0,10,1,1,2448.0,0
2917,62.0,10441,3,2,5,5,1992,0.0,3,4,3,3,337.0,1,575.0,3,1,0,3,1,3,6,7,0,0,0.0,0,2,80,700,1.5,14,1,1,1882.0,32


In [125]:
data_predictors['GarageCars'] = data_predictors['GarageCars'].astype('int64')

In [126]:
toNormalize = ['LotFrontage', 'LotArea', 'YearBuilt', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', '2ndFlrSF', 'WoodDeckSF', 'Age', 'TotalSqFeet', 'TotalPorchSF', 'MiscVal', "TotBathrooms"]

In [127]:
data_predictors[toNormalize]

,LotFrontage,LotArea,YearBuilt,MasVnrArea,BsmtFinSF1,BsmtUnfSF,2ndFlrSF,WoodDeckSF,Age,TotalSqFeet,TotalPorchSF,MiscVal,TotBathrooms
0,65.0,8450,2003,196.0,706.0,150.0,854,0,5,2566.0,61,0,3.5
1,80.0,9600,1976,0.0,978.0,284.0,0,298,31,2524.0,0,0,2.5
2,68.0,11250,2001,162.0,486.0,434.0,866,0,6,2706.0,42,0,3.5
3,60.0,9550,1915,0.0,216.0,540.0,756,0,36,2473.0,307,0,2.0
4,84.0,14260,2000,350.0,655.0,490.0,1053,192,8,3343.0,84,0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,21.0,1936,1970,0.0,0.0,546.0,546,0,36,1638.0,0,0,1.5
2915,21.0,1894,1970,0.0,252.0,294.0,546,0,36,1638.0,24,0,1.5
2916,160.0,20000,1960,0.0,1224.0,0.0,0,474,10,2448.0,0,0,2.0
2917,62.0,10441,1992,0.0,337.0,575.0,0,80,14,1882.0,32,700,1.5


Reducing skewness

In [128]:
for col in toNormalize:
    col_skew = data_predictors[col].skew()
    if abs(col_skew) > 0.8:
        data_predictors[col] = np.log1p(data_predictors[col])
        print(col, col_skew)

    mean = data_predictors[col].mean()
    std = data_predictors[col].std()
    data_predictors[col] = (data_predictors[col] - mean) / std


LotFrontage 1.1030786133465609
LotArea 13.19450466113443
MasVnrArea 2.5958670912227526
BsmtFinSF1 0.8384979504267194
BsmtUnfSF 0.9213232637256037
2ndFlrSF 0.8614823621384978
WoodDeckSF 1.8455349471713647
TotalPorchSF 2.24764564076291
MiscVal 22.190883953680686


In [129]:
for col in toNormalize:
    print(col, data_predictors[col].skew())

LotFrontage -1.0720989344397236
LotArea -0.5407766304821674
YearBuilt -0.5992170090707281
MasVnrArea 0.5392030410639009
BsmtFinSF1 -0.6167655201428037
BsmtUnfSF -2.1548547694955196
2ndFlrSF 0.3061832012668551
WoodDeckSF 0.16016034679096697
Age 0.4492335582660393
TotalSqFeet 0.7375586359198772
TotalPorchSF -0.5712816072431125
MiscVal 5.224200444713828
TotBathrooms 0.48639982899566264


In [130]:
numeric_cols = data_predictors.select_dtypes(include=np.number).columns
notToNormalize = [col for col in numeric_cols if col not in toNormalize]
data_predictors[notToNormalize]

,LotShape,LandSlope,OverallQual,OverallCond,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType2,HeatingQC,CentralAir,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,FireplaceQu,GarageFinish,GarageCars,GarageQual,PavedDrive,IsNew,NeighRich
0,3,2,7,5,3,4,3,1,1,5,1,3,1,4,8,7,0,2,2,3,2,1,2
1,3,2,6,8,3,4,3,4,1,5,1,3,1,3,6,7,3,2,2,3,2,1,2
2,2,2,7,5,3,4,3,2,1,5,1,3,1,4,6,7,3,2,2,3,2,1,2
3,2,2,7,5,3,3,4,1,1,4,1,3,1,4,7,7,4,1,3,3,2,1,2
4,2,2,8,5,3,4,3,3,1,5,1,4,1,4,9,7,3,2,3,3,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,3,2,4,7,3,3,3,1,1,4,1,3,1,3,5,7,0,0,0,0,2,1,1
2915,3,2,4,5,3,3,3,1,1,3,1,3,1,3,6,7,0,1,1,3,2,1,1
2916,3,2,5,7,3,3,3,1,1,5,1,4,1,3,7,7,3,1,2,3,2,1,1
2917,3,2,5,5,3,4,3,3,1,3,1,3,1,3,6,7,0,0,0,0,2,1,1


In [131]:
data_predictors.select_dtypes(include='category')

,MSSubClass,MSZoning,LandContour,LotConfig,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,Exterior1st,Exterior2nd,Foundation,Electrical,GarageType,MoSold,SaleType,SaleCondition
0,60,RL,Lvl,Inside,Norm,Norm,1Fam,2Story,Gable,VinylSd,VinylSd,PConc,SBrkr,Attchd,2,WD,Normal
1,20,RL,Lvl,FR2,Feedr,Norm,1Fam,1Story,Gable,MetalSd,MetalSd,CBlock,SBrkr,Attchd,5,WD,Normal
2,60,RL,Lvl,Inside,Norm,Norm,1Fam,2Story,Gable,VinylSd,VinylSd,PConc,SBrkr,Attchd,9,WD,Normal
3,70,RL,Lvl,Corner,Norm,Norm,1Fam,2Story,Gable,WdSdng,WdShing,BrkTil,SBrkr,Detchd,2,WD,Abnorml
4,60,RL,Lvl,FR2,Norm,Norm,1Fam,2Story,Gable,VinylSd,VinylSd,PConc,SBrkr,Attchd,12,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160,RM,Lvl,Inside,Norm,Norm,TwnhsE,2Story,Gable,CemntBd,CemntBd,CBlock,SBrkr,No Garage,6,WD,Normal
2915,160,RM,Lvl,Inside,Norm,Norm,TwnhsE,2Story,Gable,CemntBd,CemntBd,CBlock,SBrkr,CarPort,4,WD,Abnorml
2916,20,RL,Lvl,Inside,Norm,Norm,1Fam,1Story,Gable,VinylSd,VinylSd,CBlock,SBrkr,Detchd,9,WD,Abnorml
2917,85,RL,Lvl,Inside,Norm,Norm,1Fam,SFoyer,Gable,HdBoard,WdShing,PConc,SBrkr,No Garage,7,WD,Normal


In [132]:
cat_cols = data_predictors.select_dtypes(include='category').columns
data_predictors = pd.get_dummies(data_predictors, columns=cat_cols, drop_first=True)
data_predictors


,LotFrontage,LotArea,LotShape,LandSlope,OverallQual,OverallCond,YearBuilt,MasVnrArea,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinType2,BsmtUnfSF,HeatingQC,CentralAir,2ndFlrSF,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,FireplaceQu,GarageFinish,GarageCars,GarageQual,PavedDrive,WoodDeckSF,MiscVal,TotBathrooms,Age,IsNew,NeighRich,TotalSqFeet,TotalPorchSF,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2FmCon,BldgType_Duplx,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_WdSdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_BrkComm,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CemntBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_WdSdng,Exterior2nd_WdShing,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_No Garage,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-0.049344,-0.100872,3,2,7,5,1.047551,1.225748,3,4,3,1,0.785141,1,-0.324543,5,1,1.197504,3,1,4,8,7,0,2,2,3,2,-0.943966,-0.188613,1.592049,-0.888239,1,2,0.027822,0.434844,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False
1,0.575782,0.150607,3,2,6,8,0.156008,-0.792394,3,4,3,4,0.894347,1,0.015141,5,1,-0.863035,3,1,3,6,7,3,2,2,3,2,1.255649,-0.188613,0.351922,0.356497,1,2,-0.026778,-1.453092,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,Fa

In [133]:
from sklearn.feature_selection import VarianceThreshold

Deleting features with low variance

In [134]:
selector = VarianceThreshold(threshold=0.01)
data_reduced = selector.fit_transform(data_predictors)

cols_kept = data_predictors.columns[selector.get_support()]
data_reduced = pd.DataFrame(data_reduced, columns=cols_kept)

In [135]:
data_reduced

,LotFrontage,LotArea,LotShape,LandSlope,OverallQual,OverallCond,YearBuilt,MasVnrArea,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinType2,BsmtUnfSF,HeatingQC,CentralAir,2ndFlrSF,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,FireplaceQu,GarageFinish,GarageCars,GarageQual,PavedDrive,WoodDeckSF,MiscVal,TotBathrooms,Age,IsNew,NeighRich,TotalSqFeet,TotalPorchSF,MSSubClass_30,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_160,MSSubClass_190,MSZoning_FV,MSZoning_RL,MSZoning_RM,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_Inside,Condition1_Feedr,Condition1_Norm,Condition1_PosN,Condition1_RRAn,BldgType_2FmCon,BldgType_Duplx,BldgType_TwnhsE,HouseStyle_1Story,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Hip,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_WdSdng,Exterior1st_WdShing,Exterior2nd_BrkFace,Exterior2nd_CemntBd,Exterior2nd_HdBoard,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_WdSdng,Exterior2nd_WdShing,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Electrical_FuseF,Electrical_SBrkr,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_Detchd,GarageType_No Garage,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,SaleType_New,SaleType_WD,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-0.049344,-0.100872,3.0,2.0,7.0,5.0,1.047551,1.225748,3.0,4.0,3.0,1.0,0.785141,1.0,-0.324543,5.0,1.0,1.197504,3.0,1.0,4.0,8.0,7.0,0.0,2.0,2.0,3.0,2.0,-0.943966,-0.188613,1.592049,-0.888239,1.0,2.0,0.027822,0.434844,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.575782,0.150607,3.0,2.0,6.0,8.0,0.156008,-0.792394,3.0,4.0,3.0,4.0,0.894347,1.0,0.015141,5.0,1.0,-0.863035,3.0,1.0,3.0,6.0,7.0,3.0,2.0,2.0,3.0,2.0,1.255649,-0.188613,0.351922,0.356497,1.0,2.0,-0.026778,-1.453092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.086343,0.463206,2.0,2.0,7.0,5.0,0.981511,1.153378,3.0,4.0,3.0,2.0,0.660076,1.0,0.241267,5.0,1.0,1.201758,3.0,1.0,4.0,6.0,7.0,3.0,2.0,2.0,3.0,2.0,-0.943966,-0.188613,1.592049,-0.840364,1.0,2.0,0.209819,0.267449,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,-0.289820,0.140315,2.0,2.0,7.0,5.0,-1.858221,-0.792394,3.0,3.0,4.0,1.0,0.388866,1.0,0.357884,4.0,1.0,1.160348,3.0,1.0,4.0,7.0,7.0,4.0,1.0,3.0,3.0,2.0,-0.943966,-0.188613,-0.268141,0.595870,1.0,2.0,-0.093077,1.168113,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.722916,0.930500,2.0,2.0,8.0,5.0,0.948491,1.446380,3.0,4.0,3.0,3.0,0.760022,1.0,0.306026,5.0,1.0,1.261369,4.0,1.0,4.0,9.0,7.0,3.0,2.0,3.0,3.0,2.0,1.086734,-0.188613,1.592049,-0.744615,1.0,3.0,1.037908,0.579176,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,

In [136]:
data_model = pd.concat([data_reduced, data_target.reset_index(drop=True)], axis=1)


In [138]:
data_model.to_csv(r'D:\University\semestr_7\Machine_Learning\lab1\data\data_model.csv', index=False)

# Modeling

Написати код у Python / R, який реалізований з використанням двох (для отримання
балу не вище 75%) чи трьох (на максимальний бал) підходів на вибір:
1) SciKit learn
2) XGBoost
3) Vanilla Python (Numpy/Pandas)  

Для отримання максимального балу за лабораторну роботу заборонено використовувати  
бібліотеки з вже реалізованими аналогічними алгоритмами (потрібно реалізовувати алгоритми  
самостійно). Вбудовані алгоритми використовувати лише для порівняння власно-
запрограмованого алгоритму.

# Results Analysis

1. Вибір оптимальних параметрів регресій, їх обгрунтування
2. Оцінка помилок на начальній та тестовій вибірках
3. Порівняння результатів різних підходів (співпали чи ні, причини чому могли не
співпасти, runtime тощо)
4. Порівняти результати з аналогічними результатами, які отримані в результаті
використання вбудованих функцій
5. Результати оформити протоколом